# Somethin' Fishy - Fishing and Optimal Control
    Xander de la Bruere
    Jakob Gertsch
    Sam Layton
    Matt Mella
    Wilson Stoddard
    

## Abstract

We investigate the use of optimal control to maximize the fish harvested by a large fleet of fishing ships. In particular, we seek to use Pontryagin’s Maximum Principle and bang-bang control theory to find the most efficient path for each ship to take given that we can control the ships’ targets.

## Background


When developing a project idea, we wanted to focus on a broad class of problems involving the control of a group of agents moving through space in order to visit points of interest. To illustrate, controlling a fleet of drones photograph potential threats on a battlefield, flying a group of helicopters to dump water on forest fires, and using drones to do do search-and-rescue are all examples of this type of problem.

This project seeks to apply the classical principles and theory of optimal control to the age-old problem of fishing. Our hope is that the results of our work can be generalized to solve optimal control problems involving the coordination of agents visiting points of interest.

Optimizing fishing harvest is of great interest for several reasons. First, the fishing industry is a significant contributor to GDP throughout the world, as evidenced by its estimated global market value of \$277 billion in 2020 [1]. Since the value that the fishing industry adds to the economy is directly related to the amount of fish caught and sold, optimizing fishing harvests is an important problem to solve for those seeking to boost national economies, not to mention the benefits to the commercial fishermen who actually work in the industry.

Second, more optimal fishing would allow more people greater access to the nutritional benefits of seafood consumption. A 2022 study found that 90\% of Americans do not consume seafood at levels recommended by the US Dietary Guidelines for Americans, and that low-income individuals cite high seafood prices as the primary barrier to higher seafood consumption [2]. Finding a way to harvest more fish using the same amount of labor and capital could increase the supply of various types of seafood, driving down prices and increasing access to the health benefits of seafood for low-income individuals.

Finally, optimized fishing can help relieve problems caused by invasive species that plague lakes and rivers throughout the world. For example, many lakes and rivers in the Southeast of the United States still suffer from invasive silver and bighead carp that entered these waters during extensive flooding in the 1970s [3]. The presence of these invasive aquatic species continues to compromise the water quality and aquatic biodiversity of these waters today. Learning to effectively harvest and dispose of these fish would thus ease or perhaps completely remove the strain on these important ecosystems.

Notwithstanding the impact of optimized fishing on economic well-being, health, and aquatic ecosystems  we were surprised to find that no research has focused on the problem of optimizing group fishing efforts, leaving commercial fishermen to solve this problem on their own. While research exists for optimizing fish harvest in fisheries [4], and while other research has investigated the role of optimal control in maneuvering at sea [5], no extant research attempts addresses the issue of optimizing fishing harvest at sea. This analysis seeks to fill this gap in the literature by applying optimal control theory to maximizing the harvest of a group of fishing boats at sea.

![This is a PNG image.](visualizations/fishing_environment.png)


## Mathematical Representation

To model our state equation, we first define a scalar Gaussian vision for each ship as $g_{ij} = e^{-\alpha \lVert f_j - x_i \rVert^2}$. We then point in the direction from the ship to fishing ground $j$ with vector $d_{ij} = \frac{f_j - x_i}{\lVert f_j - x_i \rVert}$. The pull (attraction) of fishing ground $j$ on ship $i$ is vector $A_{ij} = u_{ij}g_{ij}d_{ij}F_j$, where we toggle the attractiveness of this fishing ground for ship $i$ with our control $u_{ij}$. Note that $F_j$ is the current amount of fish remaining at ground $j$, and so this directly scales the attractiveness as well. When we sum all fishing ground influences, it produces a vector sum $s_i = \sum_{j=1}^{K}A_{ij}$.\\


Now we define our first-order state equations $\dot{y} \in \mathbb{R}^{2N+K}$ as a concatenation of both:
\begin{align}
\dot{x_i} &= u_{i,k+1}\frac{s_i}{\lVert s_i \rVert} \in \large \mathbb{R}^2 \text{, and} \\
\dot{F_j} &= -rF_j\sum_{i=1}^{N}e^{-\sigma \lVert f_j - x_i \rVert^2} \in \large \mathbb{R}
\end{align}
So that we can express $\dot{y}^{\top}$ as:
\begin{equation}
\dot{y}^{\top} = \left[\dot{x}_{11}, \dot{x}_{12}, \dot{x}_{21}, \dot{x}_{22}, \dot{x}_{31}, \dot{x}_{32}, \ldots, \dot{x}_{N1}, \dot{x}_{N2}, \dot{F}_1, \dot{F}_2, \ldots, \dot{F}_K \right]
\end{equation}

We also observe starting conditions of $x_i(0) = h$ and $F_j(0) = F_0$, while $x_i(t_f)$ and $F_j(t_f)$ are free. Given our well-defined state equations, we now proceed to establish our control. We construct the functional in Bolza form as:

\begin{equation}
J[u] = \int_{0}^{tf} \left(\beta \sum_{i=1}^{N}\sum_{l=1}^{N} \frac{1}{\varepsilon +\lVert x_l - x_i \rVert^2 } + \gamma \sum_{i=1}^{N} \lVert \dot{x_i} \rVert\right)dt + \delta\sum_{j=1}^{K}F_j(t_f)
\end{equation}

We construct co-state equation from $p_i \in \mathbb{R}^2$ corresponding to $x_i$ and $p_{F_j}$ corresponding to $F_j$. Looking at endpoint conditions, we know that $y$ is free, thus we let $p_i(t_f) = 0$. Due to our functional taking the Bolza form, we also construct our endpoint condition by taking the derivative with respect to $y$ to get $p_{F_j}(t_f) = -\delta$. We also take Hamiltonian: 
\begin{equation}
H = \sum_{i=1}^{N}p_i \cdot \dot{x_i} + \sum_{j=1}^{K}p_{F_j}\dot{F}_j - \beta \sum_{i=1}^{N}\sum_{l=1}^{N} \frac{1}{\varepsilon +\lVert x_l - x_i \rVert^2 } - \gamma \sum_{i=1}^{N} \lVert \dot{x_i} \rVert
\end{equation}
Given our Hamiltonian is defined, we can derive our optimal control for both $u_{ij}$ (the toggle on the fishing ground attraction) and $u_{i,k+1}$ (the speed control of ship $i$). Taking $H_{u_{ij}} = 0$ implies our first condition:
 $$p_i^{T}(d_{ij} - \text{proj}_{s_i}d_{ij}) = 0$$

And taking $H_{u_{i,k+1}} = 0$ implies a "bang-bang" solution for speed where:
$$u_{i,k+1} = 
\begin{cases} 
V & \text{if } \frac{p_i^{T}s_i}{\lVert s_i \rVert} + \gamma > 0,\\
0 & \text{else}\\
\end{cases}$$
Finally, we find our co-state equations by first defining the derivative of $s_i$ with respect to $x_i$ as: $D_xs_i = \sum_{j=1}^{K}\frac{u_{ij}g_{ij}F_j}{\lVert f_j - x_i \rVert}\left(\left(\alpha + \frac{1}{\lVert f_j - x_i \rVert^2}\right)(f_j - x_i)(f_j - x_i)^{T} - I\right)$, and then substituting this expression into the following co-state equation for both $p_i$ and $p_{F_j}$


\begin{align}
\dot{p_i} &= 4\beta\sum_{l=1}^{N}\frac{x_l - x_i}{\left(\varepsilon +\lVert x_l - x_i \rVert^2 \right)^2} 
- 2r\sigma\sum_{j=1}^{K}\left(p_{F_j}e^{-\sigma \lVert f_j - x_i \rVert^2}\left(f_j - x_i\right) \right)
- u_{i,k+1}\left(I - \frac{s_is_i^T}{\lVert s_i \rVert ^2}\right)\frac{D_xs_i}{\lVert s_i \rVert}p_i \\
\dot{p}_{F_j} &= rp_{F_j}\sum_{i=1}^{N}e^{-\sigma \lVert f_j - x_i \rVert^2} %- \sum_{i=1}^{N}\frac{u_{ij}u_{i,K+1}g_{ij}}{\lVert s_i \rVert}p_i^T\left(I - \frac{s_is_i^T}{\lVert s_i \rVert ^2}\right)d_{ij}%
\end{align}

## Solution

Some text

<div>
<img src="visualizations/beta0costate.png" width="600"/>
</div>


## Interpretation

## Conclusion

## References

[1] Jannik Lindner. “The most surprising fishing industry statistics and trends in 2024,” 2023.

[2] David Love et al. “Affordability influences nutritional quality of seafood consumption among income and race/ethnicity groups in the united states,” 2022.

[3] U.S. Fish and Wildlife Service. “Invasive carp in southeastern waters,” 2021.

[4] Roland Pulch, Bernd Kugelmann. “Robust optimal control of fishing in a three competing species model,” 2018.

[5] Miele et al. “Optimal control of a ship for collision avoidance maneuvers,” 1999.

##[6] J. D. Lambert. "Numerical Methods for Ordinary Differential Systems," 1992. (TODO: Still using?)